# LIS 640 Applied Deep Learning : Convolutional Neural Networks

Let's try some real applications with Pytorch. In this section, we will directly use Pytoch functions to build a LeNet-5 model. Then we test the model on MNIST dataset. Your task is to compute the lacked `Output Size` and finish building the architecture in `problem4.py`. We will use functions like Linear, Conv2d, ReLU, MaxPool2d in Pytorch. Refer to [https://pytorch.org/docs/stable/nn.functional.html](https://pytorch.org/docs/stable/nn.functional.html) for more information.

# LeNet-5

The following Table shows the LeNet-5 model architecture and part of the output sizes.


|Layer|Output Size|
|----|----|
|$Input$|1\*28\*28|
|$Conv(C_{out}=20, K=5, P=2, S=1)$||
|$ReLU$||
|$MaxPool(K=2, S=2)$||
|$Conv(C_{out}=50, K=5, P=2, S=1)$||
|$ReLU$||
|$MaxPool(K=2, S=2)$||
|$Flatten$||
|$Linear$|500|
|$ReLU$|500|
|$Linear$|10|

In [1]:
from problem4 import LeNet

print("LeNet Model: ")
model = LeNet()
print(model)

LeNet Model: 
LeNet(
  (conv1): Conv2d(1, 20, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (conv2): Conv2d(20, 50, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (fc1): Linear(in_features=2450, out_features=500, bias=True)
  (fc2): Linear(in_features=500, out_features=10, bias=True)
  (relu): ReLU()
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
)


Let's start testing our model on MNIST dataset.

In [2]:
from problem4 import LeNet
import torch.optim as optim
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import torch.nn as nn
import torch

# hyperparameters
learning_rate = 0.01
num_epochs = 10

# load dataset
train_dataset = datasets.MNIST(root='dataset/', train=True, 
                               transform=transforms.Compose([transforms.ToTensor()]), download=True)
test_dataset = datasets.MNIST(root='dataset/', train=False, 
                              transform=transforms.Compose([transforms.ToTensor()]), download=True)

train_loader = DataLoader(dataset=train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=64)
dataset_sizes = {'train':len(train_dataset), 'test':len(test_dataset)}

# load model
model = LeNet().cuda()

# define loss
ce_loss = nn.CrossEntropyLoss()

# define optimizer
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# start training
for epoch in range(num_epochs):
    
    print('Epoch', epoch)
    running_loss = 0.0
    valid_loss = 0.0
    model.train()

    correct = 0
    for data in train_loader: # dataloaders[0] is train loader
        
        inputs, labels  = data
        batch_size = inputs.shape[0]
    
        inputs = inputs.cuda()
        labels = labels.cuda()
    
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = ce_loss(outputs, labels)
        loss.backward()
        optimizer.step()
                    
        correct += (torch.argmax(outputs,dim=1)==labels).sum().item()            
        running_loss += loss.data * batch_size

    epoch_loss = running_loss / dataset_sizes["train"]
    print('Training loss:', epoch_loss.item())
    print('Training Acc:', correct/dataset_sizes["train"])
    
    # evaluation each epoch
    model.eval()

    with torch.no_grad():
        correct = 0
        for data in test_loader:
            inputs, labels  = data
            batch_size = inputs.shape[0]

            inputs = inputs.cuda()
            labels = labels.cuda()

            outputs = model(inputs)
            optimizer.zero_grad()
            loss = ce_loss(outputs, labels)
        
            correct += (torch.argmax(outputs,dim=1)==labels).sum().item()
            valid_loss += loss.data * batch_size

        epoch_valid_loss = valid_loss / dataset_sizes["test"]
        print('Validation loss:', epoch_valid_loss.item())
        print('Validation Acc:', correct/dataset_sizes["test"])

Epoch 0
Training loss: 0.22340789437294006
Training Acc: 0.9323166666666667
Validation loss: 0.08622178435325623
Validation Acc: 0.972
Epoch 1
Training loss: 0.09765371680259705
Training Acc: 0.9710666666666666
Validation loss: 0.08063618838787079
Validation Acc: 0.9746
Epoch 2
Training loss: 0.0894680768251419
Training Acc: 0.9739333333333333
Validation loss: 0.09125841408967972
Validation Acc: 0.9744
Epoch 3
Training loss: 0.08500745892524719
Training Acc: 0.97465
Validation loss: 0.08929247409105301
Validation Acc: 0.9767
Epoch 4
Training loss: 0.0766744315624237
Training Acc: 0.9783833333333334
Validation loss: 0.08361557871103287
Validation Acc: 0.9765
Epoch 5
Training loss: 0.07315342873334885
Training Acc: 0.9794333333333334
Validation loss: 0.12178327143192291
Validation Acc: 0.9661
Epoch 6
Training loss: 0.0744248479604721
Training Acc: 0.9798333333333333
Validation loss: 0.08267689496278763
Validation Acc: 0.9781
Epoch 7
Training loss: 0.07364869117736816
Training Acc: 0.9799